## intro
! write intro
mogelijkheid om de Invloed van een variabele naar keuze te bekijken. Bijvoorbeeld beweging, slaap kwaliteit of inkomen  (of leeftijd, bmi, etc.). Dat je de factoren kan selecteren die je wilt vergelijken misschien ? Op deze manier kun je kijken wat mensen gelukkiger/ongelukkiger maakt. Met deze informatie kun je concluderen welke factoren je kan verbeteren om vooruitgang te boeken in het welzijn van mensen.

## config file maken
Maak een config file met de volgende inhoud: \
`[FILES] 
data = pad/naar/data
metadata = pad/naar/metadata`


In [1]:
# libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import hvplot.pandas

In [3]:
# config file
from configparser import ConfigParser
def read_config(config_file):
    config = ConfigParser()
    config.read(config_file)
    return config


config = read_config("config.ini")
#mulecules_file = config['FILES']['molecules']
data_path = config['FILES']['data']

## Data
Download en lees de data en de metadata.
Lees de data vervolgens in:


In [4]:
# load the data
data = pd.read_csv(data_path, header = 0)
data.head()


,GENDER,BIRTHYEAR,AGE_T1,AGE_T2,AGE_T3,ZIP_CODE,BMI_T1,WEIGHT_T1,HIP_T1,HEIGHT_T1,...,NSES_YEAR,NSES,NEIGHBOURHOOD1_T2,NEIGHBOURHOOD2_T2,NEIGHBOURHOOD3_T2,NEIGHBOURHOOD4_T2,NEIGHBOURHOOD5_T2,NEIGHBOURHOOD6_T2,MENTAL_DISORDER_T1,MENTAL_DISORDER_T2
0,2,1960,52,55,59,9903,21.9,70.0,91.0,178.5,...,2010,-0.148238,8.0,1.0,3.0,5.0,3.0,4.0,0.0,0.0
1,2,1964,48,51,57,8411,20.9,62.0,93.0,172.5,...,2010,0.357413,10.0,1.0,1.0,1.0,5.0,5.0,0.0,0.0
2,2,1951,58,62,68,9101,24.6,71.0,104.0,170.0,...,2010,-1.730366,9.0,1.0,2.0,2.0,3.0,4.0,0.0,0.0
3,1,1958,54,57,61,9761,25.0,70.5,95.0,169.5,...,2010,0.106612,8.0,2.0,1.0,1.0,5.0,5.0,0.0,0.0
4,2,1963,45,50,57,9201,24.0,61.0,95.0,159.8,...,2006,-1.004381,9.0,4.0,1.0,1.0,5.0,5.0,0.0,0.0


## !Installeer libraries
Installeer seaborn om plots te kunnen maken: `pip install seaborn` and `pip install seaborn[stats]`.


!Voor het dashboard ook de optie meegeven voor een scatterplot. Dan kan de gebruiker twee kolommen meegeven. (Neem dan hue="DEPRESSION_T1" om nog steeds depressie erbij betrokken te houden).

## Panel lay-out
Laten we beginnen met nadenken over de lay-out van panel. De vision voor de lay-out van het dashboard is voor nu: verschillende tabladen. Eentje met een scatterplot, een histogram en metadata. De scatterplot en histogram zijn allebei verdeelt in wel en niet depressief.

## hvplot
Hvplot werkt goed samen met panel, dus we gaan even testen hoe deze library werkt:

In [6]:
data.hvplot.hist("SCOR_VAL", by = "DEPRESSION_T1", stacked = False)

:NdOverlay   [DEPRESSION_T1]
   :Histogram   [SCOR_VAL]   (Count)

In de bovenstaande plot zie je scor_val (wat beweging is) in een histogram. De plot is verdeelt met groupby op depressie. De rode zijn mensen met depressie en de blauwe mensen zonder depressie. Je kan dus aflezen hieruit dat er aanzienlijk minder mensen met depressie zijn dan mesnen zonder depressie. Ook; hoe hoger de score_val gaat, hoe minder mensen. Dus meer mensen bewegen minder. \
Laten we iets anders proberen:

In [30]:
data.hvplot.hist(y = ["MWK_NO_VAL", "MWK_VAL"], bins = 50, alpha = 0.5, height = 400, by = "DEPRESSION_T1", subplots = True)
# MWK_VAL: 'Minutes of weekly physical activity on moderate and vigorous intensity level at baseline (T1)'
# MWK_NO_VAL: 'Minutes of weekly physical activity on moderate and vigorous intensity level, in leisure time and commuting domains (but not occupational) at baseline (T1)'

:NdLayout   [Variable]
   :Histogram   [MWK_VAL]   (Count)

Je ziet in de bovenstaande plot dat er nu twee grafieken zijn naast elkaar. Eentje met alle wekelijkse beweging en eentje zonder beroepsmatige beweging. De grafiek zonder beroepsmatige beweging lijkt hoger. Wat gek is. \
Laten we een scatterplot proberen te maken:

In [8]:
data.hvplot.scatter(y = "SCOR_VAL", x = "WEIGHT_T1", by = "DEPRESSION_T1")
# Mensen met depressie, minder beweging

:NdOverlay   [DEPRESSION_T1]
   :Scatter   [WEIGHT_T1]   (SCOR_VAL)

Je ziet in de scatterplot beweging tegen gewicht, gesorteerd op depressie. Je leest uit de grafiek af dat mensen met depressie gemiddeld minder bewegen. Dat is interresant. Hieruit kun je dusd afleiden dat meer bewegen mogelijk zou kunnen helpen tegen depressie.

De scatterplot wordt op de volgende manier gebruikt: \
`scatterplot = data.hvplot.scatter(y = widget_scatter_first,
                                  x = widget_scatter_second,
                                  by = widget_groupby_scat)`

De histogram wordt op de volgende maier gebruikt: \
`histplot = data.hvplot.hist(y = widget_hist_var, by = widget_groupby_hist)`


## Panel
Bij histogram kun je alleen de groupby gebruiken als er niet twee dezelfde variabelen geselecteerd zijn bij het opstarten. \
Om dit te voorkomen zet je in de config file een lijst met alle kolommen die de gebruiker mag gebruiken bij de assen en een lijst met opties die de geberuiker mag kiezen bij de groupby optie. Zorg ervoor dat ze niet met dezelfde kolom beginnen ! Om een list te krijgen uit de config gebruik je in je app.py:\
`import ast` en \
`var_list = ast.literal_eval(config['SETTINGS']['var_list'])` \
`groupby_list = ast.literal_eval(config['SETTINGS']['groupby_list'])`

Om de metadata goed te kunnen inladen moet je tussen de namen van de kolommen en de betekenis een `;` plaatsen. Op deze manier kan het in kolommen opgedeeld worden om in een tabel gezet te worden op de site.